<a href="https://colab.research.google.com/github/gyimesbalint/DL_project/blob/main/fitting_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing datasets

In [1]:
# https://drive.google.com/file/d/1pmNSD1nbYHEAiP065s4akRXHMWFs9Dqw/view?usp=sharing DBpedia train.csv
# https://drive.google.com/file/d/1mKededzdbJsWQnwsu-R_WSILYSvNEY7c/view?usp=sharing DBpedia test.csv
!pip install gdown
!gdown --id 1pmNSD1nbYHEAiP065s4akRXHMWFs9Dqw --output train.csv  #import train.csv from drive
!gdown --id 1mKededzdbJsWQnwsu-R_WSILYSvNEY7c --output test.csv   #import test.csv from drive

Downloading...
From: https://drive.google.com/uc?id=1pmNSD1nbYHEAiP065s4akRXHMWFs9Dqw
To: /content/train.csv
100% 174M/174M [00:01<00:00, 148MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mKededzdbJsWQnwsu-R_WSILYSvNEY7c
To: /content/test.csv
100% 21.8M/21.8M [00:00<00:00, 80.6MB/s]


In [2]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('train.csv', encoding='utf8',header=None) #read csv to dataframe
test_data = pd.read_csv('test.csv', encoding='utf8',header=None)   #read csv to dataframe

train_data.where(train_data[0] < 6, inplace = True)  #select first 5 categories
train_data = train_data[train_data[0].notnull()]     #remove NaN values

test_data.where(test_data[0] < 6, inplace = True)    #select first 5 categories
test_data = test_data[test_data[0].notnull()]        #remove NaN values

In [3]:
size_division = 2

training_data = train_data.where(train_data[0] == 1.0)
training_data = training_data[training_data[0].notnull()]
training_data = training_data.head(len(training_data) // size_division)
for i in range(2, 6):
    tmp = train_data.where(train_data[0] == (i * 1.0))
    tmp = tmp[tmp[0].notnull()]
    tmp = tmp.head(len(tmp) // size_division)
    training_data = pd.concat([training_data, tmp])
train_data = training_data #reduce training_data size
train_data

,0,1,2
0,1.0,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...
1,1.0,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...
2,1.0,Q-workshop,Q-workshop is a Polish company located in Poz...
3,1.0,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...
4,1.0,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...
...,...,...,...
179995,5.0,Pedro de Sousa Holstein 1st Duke of Palmela,D. Pedro de Sousa Holstein 1st Duke of Faial ...
179996,5.0,Joseph P. Kennedy II,Joseph Patrick Joe Kennedy II (born September...
179997,5.0,William Legge 1st Earl of Dartmouth,William Legge 1st Earl of Dartmouth (1672-175...
179998,5.0,Rolando Crespo,Rolando Crespo is a former Representative of ...


In [4]:
testing_data = test_data.where(test_data[0] == 1.0)
testing_data = testing_data[testing_data[0].notnull()]
testing_data = testing_data.head(len(testing_data) // size_division)
for i in range(2, 6):
    tmp = test_data.where(test_data[0] == (i * 1.0))
    tmp = tmp[tmp[0].notnull()]
    tmp = tmp.head(len(tmp) // size_division)
    testing_data = pd.concat([testing_data, tmp])
test_data = testing_data #reduce testing_data size
test_data

,0,1,2
0,1.0,TY KU,TY KU /taɪkuː/ is an American alcoholic bever...
1,1.0,Odd Lot Entertainment,OddLot Entertainment founded in 2001 by longt...
2,1.0,Henkel,Henkel AG & Company KGaA operates worldwide w...
3,1.0,GOAT Store,The GOAT Store (Games Of All Type Store) LLC ...
4,1.0,RagWing Aircraft Designs,RagWing Aircraft Designs (also called the Rag...
...,...,...,...
22495,5.0,Louis Coppersmith,W. Louis Coppersmith (June 19 1928 – January ...
22496,5.0,Jeff Johnson (Minnesota politician),Jeff Johnson (born 1966) is a politician from...
22497,5.0,Carlos Trujillo,Carlos Trujillo (born on February 25 1983) is...
22498,5.0,Charles Melvin Price,Charles Melvin Price (January 1 1905 - April ...


In [5]:
train_data.sample(5) #sample from train data

,0,1,2
137430,4.0,Kristian Platt,Kristian Platt (born 15 December 1991 Rock Fe...
135642,4.0,Vedran Ćorluka,Vedran Ćorluka (pronounced [ʋědran t͡ɕǒrluka]...
49904,2.0,Vainona High School,Vainona High School is a high school in a plu...
83420,3.0,Chloé Sainte-Marie,Chloé Sainte-Marie (born Marie-Aline Joyal on...
43793,2.0,Uppsala University,Uppsala University (Swedish: Uppsala universi...


In [6]:
test_data.sample(5) #sample from test data

,0,1,2
2065,1.0,Arts Marketing Association,The Arts Marketing Association (AMA) is a pro...
6332,2.0,Mewar University,Mewar University is an autonomous body set up...
12367,3.0,Hi-Yo Silver!,Hi-Yo Silver! was a musical solo project by B...
21494,5.0,Ashraf Abdelwahab,Ashraf Hassan Abdelwahab (Arabic: أشرف حسن عب...
1897,1.0,Lockheed Corporation,The Lockheed Corporation (originally Loughead...


In [7]:
train_label = pd.to_numeric(train_data.iloc[:,0]) #select labels (int) from train data
train_text = train_data.iloc[:,1:3] #select article text from train data
train_label.shape

(100000,)

In [8]:
test_label = pd.to_numeric(test_data.iloc[:,0]) #select labels (int) from test data
test_text = test_data.iloc[:,1:3] #select article text from train data
test_label.shape

(12500,)

#Pre-processing

In [9]:
from sklearn.feature_extraction.text import CountVectorizer #vectorizer for article text data
from nltk.corpus import stopwords #stopwords for desktop usage
import nltk
nltk.download('stopwords') #stopwords for collab notebook

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
vectorizer = CountVectorizer(stop_words=stopwords.words('english'), analyzer='word', ngram_range=(1, 1)) #vectorizer for words, removing stopwords

In [11]:
test_title = test_text.iloc[:,0] #select titles from test text
test_desc = test_text.iloc[:,1]  #select descriptions from test text

In [12]:
train_title = train_text.iloc[:,0] #select titles from train text
train_desc = train_text.iloc[:,1]  #select descriptions from train text

In [13]:
titles = pd.concat([test_title, train_title]) #concat titles for vectorazition
descs = pd.concat([test_desc, train_desc]) #concat descriptions for vectorazition

In [14]:
titles = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(titles)) #vectorizing test_title and adding it to a dataframe
title_feature_names = np.asarray(vectorizer.get_feature_names()) #getting all feature names for test_title
descs = pd.DataFrame.sparse.from_spmatrix(vectorizer.fit_transform(descs)) #vectorizing test_desc and adding it to a dataframe
desc_feature_names = np.asarray(vectorizer.get_feature_names()) #getting all feature names for test_desc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
word_rarity = 8

titles = titles.loc[:, titles.sum(axis=0) > word_rarity] #remove extremely rare words

In [16]:
titles.shape

(112500, 3948)

In [17]:
descs = descs.loc[:, descs.sum(axis=0) > word_rarity] #remove extremely rare words

In [18]:
descs.shape

(112500, 23472)

In [19]:
test_title = titles.head(len(test_title.index)) #split vectorized data
test_title.shape

(12500, 3948)

In [20]:
#First 5 rows and feature names obtained with the vectoritzer
print(test_title[0:5])
print(title_feature_names[0:5])

   232    236    237    238    239    ...  77371  77372  77385  77508  77638
0      0      0      0      0      0  ...      0      0      0      0      0
1      0      0      0      0      0  ...      0      0      0      0      0
2      0      0      0      0      0  ...      0      0      0      0      0
3      0      0      0      0      0  ...      0      0      0      0      0
4      0      0      0      0      0  ...      0      0      0      0      0

[5 rows x 3948 columns]
['002' '07' '0verflow' '10' '100']


In [21]:
test_desc = descs.head(len(test_desc.index))

In [22]:
test_desc.shape

(12500, 23472)

In [23]:
#First 5 rows and feature names obtained with the vectorizer
print(test_desc[0:5])

   0       1       47      63      ...  200778  201952  201965  202410
0       0       0       0       0  ...       0       0       0       0
1       0       0       0       0  ...       0       0       0       0
2       0       0       0       0  ...       0       0       0       0
3       0       0       0       0  ...       0       0       0       0
4       0       0       0       0  ...       0       0       0       0

[5 rows x 23472 columns]


In [24]:
train_title = titles.tail(len(titles) - len(test_title)) #split vectorized data

In [25]:
train_title.shape

(100000, 3948)

In [26]:
#First 5 rows and feature names obtained with the vectoritzer
print(train_title[0:5])

       232    236    237    238    239    ...  77371  77372  77385  77508  77638
12500      0      0      0      0      0  ...      0      0      0      0      0
12501      0      0      0      0      0  ...      0      0      0      0      0
12502      0      0      0      0      0  ...      0      0      0      0      0
12503      0      0      0      0      0  ...      0      0      0      0      0
12504      0      0      0      0      0  ...      0      0      0      0      0

[5 rows x 3948 columns]


In [27]:
train_desc = descs.tail(len(descs) - len(test_desc.index))

In [28]:
train_desc.shape

(100000, 23472)

In [29]:
#First 5 rows and feature names obtained with the vectoritzer
print(train_desc[0:5])

       0       1       47      63      ...  200778  201952  201965  202410
12500       0       0       0       0  ...       0       0       0       0
12501       0       0       0       0  ...       0       0       0       0
12502       0       0       0       0  ...       0       0       0       0
12503       0       0       0       0  ...       0       0       0       0
12504       0       0       0       0  ...       0       0       0       0

[5 rows x 23472 columns]


In [30]:
from scipy.sparse import csr_matrix #for min-max scaling sparse matrices

In [31]:
def normalize(df): #function for min-max scaling of dataframes
    result = df.copy()
    for feature_name in df.columns:
        max_value = csr_matrix(df[feature_name]).max()
        min_value = csr_matrix(df[feature_name]).min()
        result[feature_name] = (df[feature_name] - min_value) / max((max_value - min_value), 1)
    return result

In [32]:
test_title = normalize(test_title) #min-max scale test_title

In [33]:
print(test_title[0:5])

   232    236    237    238    239    ...  77371  77372  77385  77508  77638
0    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
1    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
2    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
3    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
4    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0

[5 rows x 3948 columns]


In [34]:
test_desc = normalize(test_desc) #min-max scale test_desc

In [35]:
print(test_desc[0:5])

   0       1       47      63      ...  200778  201952  201965  202410
0     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
1     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
2     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
3     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 23472 columns]


In [36]:
train_title = normalize(train_title) #min-max scale train_title

In [37]:
print(train_title[0:5])

       232    236    237    238    239    ...  77371  77372  77385  77508  77638
12500    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
12501    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
12502    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
12503    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0
12504    0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0

[5 rows x 3948 columns]


In [38]:
train_desc = normalize(train_desc) #min-max scale train_desc

In [39]:
print(train_desc[0:5])

       0       1       47      63      ...  200778  201952  201965  202410
12500     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
12501     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
12502     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
12503     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
12504     0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 23472 columns]


In [40]:
train_set = pd.concat([train_title, train_desc], axis=1)

input_shape = train_set.iloc[0].shape
input_shape #get input_shape for the model

(27420,)

In [41]:
from sklearn.model_selection import train_test_split

test_set = pd.concat([test_title, test_desc], axis=1)
test_size = 0.5
val_set, test_set, val_label, test_label = train_test_split(test_set, test_label, test_size=test_size) #create validation sets

In [42]:
from tensorflow.keras.utils import to_categorical
val_set = val_set.to_numpy()

val_label = val_label.to_numpy()
val_label = val_label - 1
val_label = to_categorical(val_label, num_classes=5) #reshape val_label from int to categorical
val_label

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [43]:
test_set = test_set.to_numpy()

test_label = test_label.to_numpy()
test_label = test_label - 1
test_label = to_categorical(test_label, num_classes=5) #reshape test_label from int to categorical
test_label

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

Our **X** will be the sparse matrices and the feature names from the title and the description, and **Y** will be the labels provided in the csv file.

# Data generator

In [44]:
from tensorflow.keras.utils import Sequence, to_categorical

class DataGenerator(Sequence): #custom data generator
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=4, dim=(input_shape[0]), n_channels=1,
                 n_classes=5, shuffle=True):
        #Initialization
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        #Generate one batch of data
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        #Updates indexes after each epoch
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        #Generates data containing batch_size samples
        # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim))#, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i in range(len(list_IDs_temp)):
            # Store sample
            X[i] = self.list_IDs.iloc[list_IDs_temp[i]].to_numpy()#.reshape(self.dim, self.n_channels)

            # Store class
            y[i] = self.labels.iloc[list_IDs_temp[i]]

        return X, to_categorical(y - 1, num_classes=self.n_classes)

In [45]:
train_generator = DataGenerator(train_set, train_label)

#Hyperparameter tuning

In [46]:
%tensorflow_version 2.x

In [47]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.9 MB/s 


In [48]:
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
#from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [49]:
def build_model(hp): # model builder for the tuner
  model = keras.Sequential([
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128,step=16),
        activation='relu',
        input_shape=input_shape
    ),
     keras.layers.Flatten(),
    #keras.layers.Dense(
    #    units=hp.Int('dense_2_units', min_value=64, max_value=128,step=16),
    #    activation='relu'
    #),
    #keras.layers.Dropout(hp.Float('dropout_1',min_value=0.25, max_value=0.75, step=0.25)),
    keras.layers.Dense(
        units=hp.Int('dense_2_units', min_value=64, max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dropout(hp.Float('dropout_1',min_value=0.25, max_value=0.75, step=0.25)),
    keras.layers.Dense(5, activation='softmax')
  ])


  model.compile(
      loss='categorical_crossentropy',
      optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
      metrics=['accuracy'])
  
  return model

In [50]:
from kerastuner.tuners import Hyperband

tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    factor=3,
    max_epochs=10,
    directory='output',
    project_name='MLP_model1')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [51]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
dense_2_units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 16, 'sampling': None}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.25, 'max_value': 0.75, 'step': 0.25, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [ ]:
tuner.search(train_generator, epochs=10, steps_per_epoch=250, validation_data=(val_set,val_label))

Trial 2 Complete [00h 13m 55s]
val_accuracy: 0.9607999920845032

Best val_accuracy So Far: 0.9607999920845032
Total elapsed time: 00h 28m 19s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
dense_1_units     |96                |96                
dense_2_units     |112               |80                
dropout_1         |0.75              |0.75              
learning_rate     |0.01              |0.001             
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/2
 74/250 [=======>......................] - ETA: 4:48 - loss: 1.2012 - accuracy: 0.4595

In [ ]:
tuner.results_summary()

In [ ]:
params_best = tuner.get_best_hyperparameters(num_trials=1)[0] # Parameters of the best model
params_best.get_config()['values']

# Model and fitting

In [ ]:
model = tuner.hypermodel.build(params_best) # Building a model with the best parameters

In [ ]:
print(model.summary())

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard #set up callbacks
patience=10
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='model.hdf5', save_best_only=True, verbose=1)
tb = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=1)

In [ ]:
#train model
network_history = model.fit(train_generator, validation_data=(val_set, val_label), epochs=20,steps_per_epoch=250, verbose=1, callbacks=[early_stopping, checkpointer, tb])

# Evaluation

In [ ]:
from keras.models import load_model
model.load_weights("model.hdf5")
test_err = model.evaluate(test_set, test_label)
print("Test error:", test_err[0], "Teszt accuracy:", test_err[1])

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
# get predictions
y_pred = model.predict(test_set)
y_pred = np.argmax(y_pred,1)
y_true = np.argmax(test_label,1)

print("test accuracy: %g" %(accuracy_score(y_true, y_pred)))
print("Precision", precision_score(y_true, y_pred, average="macro"))
print("Recall", recall_score(y_true, y_pred, average="macro"))
print("f1_score", f1_score(y_true, y_pred, average="macro"))
print("\nConfusion matrix: ")
conf=confusion_matrix(y_true, y_pred)
print(conf)

In [ ]:
import seaborn as sns
sns.heatmap(conf, annot=True, fmt='d', vmax=20) # get heatmap